<a href="https://colab.research.google.com/github/alexmjn/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [26]:
df.columns

Index(['bathrooms', 'bedrooms', 'created', 'description', 'display_address',
       'latitude', 'longitude', 'price', 'street_address', 'interest_level',
       'elevator', 'cats_allowed', 'hardwood_floors', 'dogs_allowed',
       'doorman', 'dishwasher', 'no_fee', 'laundry_in_building',
       'fitness_center', 'pre-war', 'laundry_in_unit', 'roof_deck',
       'outdoor_space', 'dining_room', 'high_speed_internet', 'balcony',
       'swimming_pool', 'new_construction', 'terrace', 'exclusive', 'loft',
       'garden_patio', 'wheelchair_access', 'common_outdoor_space'],
      dtype='object')

In [27]:
df.dtypes

bathrooms               float64
bedrooms                  int64
created                  object
description              object
display_address          object
latitude                float64
longitude               float64
price                     int64
street_address           object
interest_level           object
elevator                  int64
cats_allowed              int64
hardwood_floors           int64
dogs_allowed              int64
doorman                   int64
dishwasher                int64
no_fee                    int64
laundry_in_building       int64
fitness_center            int64
pre-war                   int64
laundry_in_unit           int64
roof_deck                 int64
outdoor_space             int64
dining_room               int64
high_speed_internet       int64
balcony                   int64
swimming_pool             int64
new_construction          int64
terrace                   int64
exclusive                 int64
loft                      int64
garden_p

We have our target column of price, as well as a set of integer constants of varying theoretical relevance. We also have some "object" columns that we may want to convert into features that can be processed. 

In [6]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [0]:
df["pet_friendliness"] = df["cats_allowed"] + df["dogs_allowed"]
df["num_amenities"] = df['elevator'] + df["hardwood_floors"] + df["doorman"] + df["dishwasher"] + df["laundry_in_building"] + df["fitness_center"] + df["roof_deck"] + df["laundry_in_unit"]
df["interest_level"] = df["interest_level"].replace({"high" : 3,
                                                     "medium" : 2,
                                                     "low" : 1})

In [39]:
df["description_len"] = df["description"].apply(len)
df["description_len"].head()

0    588
1      8
2    691
3    492
4    479
Name: description_len, dtype: int64

In [0]:
df["month"] = pd.to_datetime(df["created"], infer_datetime_format=True).dt.month

In [0]:
train = df[df["month"] <= 5]
test = df[df["month"] == 6]

In [42]:
df.corr()

,bathrooms,bedrooms,latitude,longitude,price,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,pet_friendliness,num_amenities,description_len,month
bathrooms,1.000000,0.524082,0.013407,-0.020099,0.687296,-0.072246,0.132882,0.021475,0.096922,0.024539,0.157173,0.172230,0.129547,-0.013202,0.148334,-0.016214,0.209366,0.102113,0.144450,0.233038,0.089073,0.136990,0.113315,0.066826,0.140767,-0.001944,0.014260,0.096283,0.066940,-0.003403,0.023373,0.216005,0.141108,-0.000452
bedrooms,0.524082,1.000000,0.003490,0.055117,0.535503,0.040735,-0.024821,-0.011173,0.096420,-0.009750,-0.047562,0.152102,0.163000,0.008558,0.013150,0.001420,0.151790,0.045451,0.124912,0.189415,0.061006,0.097772,0.033441,0.000907,0.099149,-0.014880,-0.107341,0.070760,0.012306,0.003234,-0.010631,0.079924,0.103345,0.014331
latitude,0.013407,0.003490,1.000000,0.329185,-0.036286,-0.046203,-0.010523,-0.029808,0.018653,-0.030954,-0.043393,-0.020612,-0.026788,-0.041635,-0.108455,0.026802,-0.044339,-0.063198,-0.079919,0.017146,-0.030969,0.016877,0.027020,-0.056093,0.006466,-0.060054,-0.015650,-0.000589,-0.068330,-0.117199,-0.030868,-0.058574,-0.042333,0.012899
longitude,-0.020099,0.055117,0.329185,1.000000,-0.251004,0.059222,-0.189836,-0.058475,-0.108493,-0.070329,-0.275734,-0.169220,-0.088033,-0.044562,-0.254960,0.000196,-0.130139,-0.161466,-0.098595,-0.024793,-0.125635,-0.035474,-0.075046,-0.108001,-0.049016,0.046755,-0.060018,-0.029846,-0.063635,-0.102955,-0.065423,-0.283252,-0.104080,0.010717
price,0.687296,0.535503,-0.036286,-0.251004,1.000000,-0.203596,0.207169,0.051453,0.101503,0.060401,0.276215,0.223899,0.132240,-0.019417,0.228775,-0.029122,0.271195,0.122929,0.142146,0.242911,0.090269,0.139140,0.134513,0.071431,0.145973,-0.013251,0.007100,0.103672,0.072517,0.011517,0.056815,0.305622,0.161948,0.009466
interest_level,-0.072246,0.040735,-0.046203,0.059222,-0.203596,1.000000,-0.008542,-0.057777,0.118992,-0.064214,-0.078467,0.064014,0.139567,0.100112,-0.042343,-0.038278,0.015665,0.014348,0.071750,0.044167,0.041725,0.027028,-0.001108,0.015816,0.028811,0.016218,0.020238,0.029311,0.011676,0.029864,-0.061968,0.029648,0.058944,-0.005012
elevator,0.132882,-0.024821,-0.010523,-0.189836,0.207169,-0.008542,1.000000,0.033347,0.270831,0.034833,0.614558,0.349832,0.227895,0.141097,0.431070,-0.097015,0.134158,0.332028,0.204343,0.200591,0.277666,0.168081,0.183664,0.184178,0.135329,0.025895,0.054918,0.084056,0.155396,0.114882,0.034636,0.711572,0.230818,-0.009841
cats_allowed,0.021475,-0.011173,-0.029808,-0.058475,0.051453,-0.057777,0.033347,1.000000,-0.177633,0.937245,0.088480,-0.047880,-0.024052,0.105644,0.126886,0.047500,-0.008827,0.027590,0.066713,-0.022404,0.077759,0.019979,0.009704,0.042710,0.006152,0.031414,-0.037966,0.006690,0.039656,0.104496,0.984259,0.016119,0.041551,0.006935
hardwood_floors,0.096922,0.096420,0.018653,-0.108493,0.101503,0.118992,0.270831,-0.177633,1.000000,-0.185663,0.205119,0.634983,0.342971,-0.144728,0.167510,0.013435,0.360716,0.278727,0.188913,0.317281,0.237935,0.178089,0.170589,0.187635,0.181005,-0.194436,0.116572,0.161751,0.124829,-0.120489,-0.184558,0.628725,0.282140,-0.010922
dogs_allowed,0.024539,-0.009750,-0.030954,-0.070329,0.060401,-0.064214,0.034833,0.937245,-0.185663,1.000000,0.095434,-0.043839,-0.011414,0.093035,0.131521,0.051610,0.002510,0.034104,0.067859,-0.016154,0.089425,0.024472,0.009675,0.055655,0.005219,0.032729,-0.041014,0.009229,0.048123,0.106071,0.984113,0.019884,0.050106,0.007475


In [0]:
features = ["bathrooms", "bedrooms", "description_len", "pet_friendliness", "num_amenities", "interest_level", "pre-war", "dining_room", "loft"]
target = "price"

In [0]:
model = LinearRegression()

In [0]:
X_train = train[features]
X_test = test[features]
y_train = train[target]
y_test = test[target]

In [48]:
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [49]:
print('Intercept', model.intercept_)
coefficients = pd.Series(model.coef_, features)
print(coefficients.to_string())

Intercept 1027.2586964041643
bathrooms           1800.125218
bedrooms             435.694536
description_len       -0.001767
pet_friendliness      60.560608
num_amenities        147.928123
interest_level      -513.357743
pre-war             -103.747938
dining_room          131.817016
loft                 164.236846


Of note, the length of description doesn't seem to add much at all. I wonder what the interest level is capturing -- perhaps people are disproportionately interested in properties that are cheaper than would've been predicted by a simplistic beds/baths/amenities model, and thus interest level negatively "predicts" price.

If I were trying to model a causal relationship between X variables and the target price, I'd omit interest level -- there's no plausible way a higher interest level could cause a lower price. 

I believe neighborhood is likely a variable of interest. I wonder what the easiest way to capture neighborhood based on longitude and latitude would be. Could clustering offer something here? 

I suspect a non-linear-regression model (neural network or random forest) might do better at iteratively and automatically capturing these patterns; raw correlation clearly shouldn't tell us much here.

In [50]:
y_pred = model.predict(X_train)
rmse = mean_squared_error(y_train, y_pred, squared = False)
mae = mean_absolute_error(y_train, y_pred)
r_squared = r2_score(y_train, y_pred)
print(rmse, mae, r_squared)

1147.976485901253 750.4692554173773 0.5755755118138028


In [0]:
def generate_tests(X, y, model):
  y_pred = model.predict(X)
  rmse = mean_squared_error(y, y_pred, squared = False)
  mae = mean_absolute_error(y, y_pred)
  r_squared = r2_score(y, y_pred)
  return rmse, mae, r_squared

In [52]:
generate_tests(X_train, y_train, model)

(1147.976485901253, 750.4692554173773, 0.5755755118138028)

In [55]:
generate_tests(X_test, y_test, model)

(1133.5719459125687, 750.644208005171, 0.5865583772379243)